<a href="https://colab.research.google.com/github/TonyWang126/NYU-Econometrics-2021/blob/main/Note_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

Story Background: https://www.gemini.com/cryptopedia/the-dao-hack-makerdao

Event: July 20, 2016, at block 192,000

[Data Metrics](https://github.com/coinmetrics-io/data/blob/master/csv/metrics.csv)

In [ ]:
import pandas as pd
import numpy as np

###import the data for Ethereum

In [ ]:
df_eth=pd.read_csv("https://raw.githubusercontent.com/coinmetrics-io/data/master/csv/eth.csv")
df_eth.head()

,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,AdrBalNtv0.001Cnt,AdrBalNtv0.01Cnt,AdrBalNtv0.1Cnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,AdrBalNtv1Cnt,AdrBalNtv1KCnt,AdrBalNtv1MCnt,AdrBalUSD100Cnt,AdrBalUSD100KCnt,AdrBalUSD10Cnt,AdrBalUSD10KCnt,AdrBalUSD10MCnt,AdrBalUSD1Cnt,AdrBalUSD1KCnt,AdrBalUSD1MCnt,AssetEODCompletionTime,BlkCnt,BlkSizeMeanByte,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktFFUSD,CapRealUSD,DiffLast,DiffMean,...,SplyAdrBal1in10M,SplyAdrBal1in1B,SplyAdrBal1in1K,SplyAdrBal1in1M,SplyAdrBalNtv0.001,SplyAdrBalNtv0.01,SplyAdrBalNtv0.1,SplyAdrBalNtv1,SplyAdrBalNtv10,SplyAdrBalNtv100,SplyAdrBalNtv100K,SplyAdrBalNtv10K,SplyAdrBalNtv1K,SplyAdrBalNtv1M,SplyAdrBalUSD1,SplyAdrBalUSD10,SplyAdrBalUSD100,SplyAdrBalUSD100K,SplyAdrBalUSD10K,SplyAdrBalUSD10M,SplyAdrBalUSD1K,SplyAdrBalUSD1M,SplyAdrTop100,SplyAdrTop10Pct,SplyAdrTop1Pct,SplyCur,SplyExpFut10yr,SplyFF,TxCnt,TxCntSec,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d
0,2015-07-30,9206.0,5685.0,9203.0,9203.0,1237.0,9151.0,9203.0,145.0,8094.0,9203.0,9203.0,9203.0,9203.0,7952.0,122.0,9142.0,1017.0,9203.0,5363.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,6911.0,643.886124,NaN,NaN,NaN,NaN,NaN,0.0,3.382792e+11,1.213355e+11,...,7.204905e+07,7.204930e+07,4.050941e+07,7.201769e+07,7.204930e+07,7.204930e+07,7.204930e+07,7.204930e+07,7.204898e+07,7.200534e+07,3.853810e+07,5.890308e+07,7.102484e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.633810e+07,5.793308e+07,3.553810e+07,7.204930e+07,4.471633e+08,7.123930e+07,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,2015-07-31,424.0,5692.0,9359.0,9359.0,1238.0,9278.0,9359.0,145.0,8152.0,9359.0,9359.0,9359.0,9359.0,8003.0,122.0,9271.0,1018.0,9359.0,5368.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,6863.0,582.027976,NaN,NaN,NaN,NaN,NaN,0.0,8.231927e+11,6.029419e+11,...,7.208510e+07,7.208549e+07,4.050941e+07,7.205138e+07,7.208549e+07,7.208549e+07,7.208549e+07,7.208549e+07,7.208504e+07,7.203841e+07,3.853810e+07,5.891693e+07,7.104616e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.633810e+07,5.808693e+07,3.563810e+07,7.208549e+07,2.721499e+08,7.127549e+07,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,2015-08-01,413.0,5699.0,9474.0,9474.0,1238.0,9368.0,9474.0,145.0,8188.0,9474.0,9474.0,9474.0,9474.0,8030.0,122.0,9360.0,1019.0,9474.0,5370.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,5293.0,575.164179,NaN,NaN,NaN,NaN,NaN,0.0,9.566506e+11,8.872542e+11,...,7.211269e+07,7.211320e+07,4.050941e+07,7.207721e+07,7.211320e+07,7.211320e+07,7.211320e+07,7.211320e+07,7.211262e+07,7.206358e+07,3.853810e+07,5.893198e+07,7.106000e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.633810e+07,5.821198e+07,3.573810e+07,7.211320e+07,2.257182e+08,7.130320e+07,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,2015-08-02,432.0,5708.0,9561.0,9561.0,1238.0,9456.0,9561.0,145.0,8223.0,9561.0,9561.0,9561.0,9561.0,8051.0,122.0,9443.0,1019.0,9561.0,5381.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,5358.0,580.878686,NaN,NaN,NaN,NaN,NaN,0.0,1.102546e+12,1.020254e+12,...,7.214092e+07,7.214142e+07,4.050941e+07,7.210380e+07,7.214142e+07,7.214142e+07,7.214142e+07,7.214142e+07,7.214080e+07,7.208884e+07,3.853810e+07,5.893877e+07,7.108533e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.633810e+07,5.830877e+07,3.583810e+07,7.214142e+07,2.100940e+08,7.133142e+07,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,2015-08-03,444.0,5715.0,9637.0,9637.0,1237.0,9520.0,9637.0,145.0,8262.0,9637.0,9637.0,9637.0,9637.0,8078.0,122.0,9508.0,1019.0,9637.0,5382.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,5280.0,587.112311,NaN,NaN,NaN,NaN,NaN,0.0,1.154732e+12,1.125837e+12,...,7.216883e+07,7.216940e+07,4.050941e+07,7.213076e+07,7.216940e+07,7.216940e+07,7.216940e+07,7.216940e+07,7.216873e+07,7.211491e+07,3.853810e+07,5.8944

In [ ]:
df_eth['time']=pd.to_datetime(df_eth['time'])
df_eth.dtypes

time                datetime64[ns]
AdrActCnt                  float64
AdrBal1in100KCnt           float64
AdrBal1in100MCnt           float64
AdrBal1in10BCnt            float64
                         ...      
TxTfrValMedNtv             float64
TxTfrValMedUSD             float64
VelCur1yr                  float64
VtyDayRet180d              float64
VtyDayRet30d               float64
Length: 139, dtype: object

In [ ]:
df_eth['Type'] = 'eth'
df_eth.head()

,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,AdrBalNtv0.001Cnt,AdrBalNtv0.01Cnt,AdrBalNtv0.1Cnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,AdrBalNtv1Cnt,AdrBalNtv1KCnt,AdrBalNtv1MCnt,AdrBalUSD100Cnt,AdrBalUSD100KCnt,AdrBalUSD10Cnt,AdrBalUSD10KCnt,AdrBalUSD10MCnt,AdrBalUSD1Cnt,AdrBalUSD1KCnt,AdrBalUSD1MCnt,AssetEODCompletionTime,BlkCnt,BlkSizeMeanByte,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktFFUSD,CapRealUSD,DiffLast,DiffMean,...,SplyAdrBal1in1B,SplyAdrBal1in1K,SplyAdrBal1in1M,SplyAdrBalNtv0.001,SplyAdrBalNtv0.01,SplyAdrBalNtv0.1,SplyAdrBalNtv1,SplyAdrBalNtv10,SplyAdrBalNtv100,SplyAdrBalNtv100K,SplyAdrBalNtv10K,SplyAdrBalNtv1K,SplyAdrBalNtv1M,SplyAdrBalUSD1,SplyAdrBalUSD10,SplyAdrBalUSD100,SplyAdrBalUSD100K,SplyAdrBalUSD10K,SplyAdrBalUSD10M,SplyAdrBalUSD1K,SplyAdrBalUSD1M,SplyAdrTop100,SplyAdrTop10Pct,SplyAdrTop1Pct,SplyCur,SplyExpFut10yr,SplyFF,TxCnt,TxCntSec,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,Type
0,2015-07-30,9206.0,5685.0,9203.0,9203.0,1237.0,9151.0,9203.0,145.0,8094.0,9203.0,9203.0,9203.0,9203.0,7952.0,122.0,9142.0,1017.0,9203.0,5363.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,6911.0,643.886124,NaN,NaN,NaN,NaN,NaN,0.0,3.382792e+11,1.213355e+11,...,7.204930e+07,4.050941e+07,7.201769e+07,7.204930e+07,7.204930e+07,7.204930e+07,7.204930e+07,7.204898e+07,7.200534e+07,3.853810e+07,5.890308e+07,7.102484e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.633810e+07,5.793308e+07,3.553810e+07,7.204930e+07,4.471633e+08,7.123930e+07,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,eth
1,2015-07-31,424.0,5692.0,9359.0,9359.0,1238.0,9278.0,9359.0,145.0,8152.0,9359.0,9359.0,9359.0,9359.0,8003.0,122.0,9271.0,1018.0,9359.0,5368.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,6863.0,582.027976,NaN,NaN,NaN,NaN,NaN,0.0,8.231927e+11,6.029419e+11,...,7.208549e+07,4.050941e+07,7.205138e+07,7.208549e+07,7.208549e+07,7.208549e+07,7.208549e+07,7.208504e+07,7.203841e+07,3.853810e+07,5.891693e+07,7.104616e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.633810e+07,5.808693e+07,3.563810e+07,7.208549e+07,2.721499e+08,7.127549e+07,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,eth
2,2015-08-01,413.0,5699.0,9474.0,9474.0,1238.0,9368.0,9474.0,145.0,8188.0,9474.0,9474.0,9474.0,9474.0,8030.0,122.0,9360.0,1019.0,9474.0,5370.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,5293.0,575.164179,NaN,NaN,NaN,NaN,NaN,0.0,9.566506e+11,8.872542e+11,...,7.211320e+07,4.050941e+07,7.207721e+07,7.211320e+07,7.211320e+07,7.211320e+07,7.211320e+07,7.211262e+07,7.206358e+07,3.853810e+07,5.893198e+07,7.106000e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.633810e+07,5.821198e+07,3.573810e+07,7.211320e+07,2.257182e+08,7.130320e+07,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,eth
3,2015-08-02,432.0,5708.0,9561.0,9561.0,1238.0,9456.0,9561.0,145.0,8223.0,9561.0,9561.0,9561.0,9561.0,8051.0,122.0,9443.0,1019.0,9561.0,5381.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,5358.0,580.878686,NaN,NaN,NaN,NaN,NaN,0.0,1.102546e+12,1.020254e+12,...,7.214142e+07,4.050941e+07,7.210380e+07,7.214142e+07,7.214142e+07,7.214142e+07,7.214142e+07,7.214080e+07,7.208884e+07,3.853810e+07,5.893877e+07,7.108533e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.633810e+07,5.830877e+07,3.583810e+07,7.214142e+07,2.100940e+08,7.133142e+07,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,eth
4,2015-08-03,444.0,5715.0,9637.0,9637.0,1237.0,9520.0,9637.0,145.0,8262.0,9637.0,9637.0,9637.0,9637.0,8078.0,122.0,9508.0,1019.0,9637.0,5382.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.614335e+09,5280.0,587.112311,NaN,NaN,NaN,NaN,NaN,0.0,1.154732e+12,1.125837e+12,...,7.216940e+07,4.050941e+07,7.213076e+07,7.216940e+07,7.216940e+07,7.216940e+07,7.216940e+07,7.216873e+07,7.211491e+07,3.853810e+07,5.894429e+07,7.110134e+07,1.290148e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### Import the Data for Ethereum Classic

In [ ]:
df_etc=pd.read_csv("https://raw.githubusercontent.com/coinmetrics-io/data/master/csv/etc.csv")
df_etc.head()

,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,AdrBalNtv0.001Cnt,AdrBalNtv0.01Cnt,AdrBalNtv0.1Cnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,AdrBalNtv1Cnt,AdrBalNtv1KCnt,AdrBalNtv1MCnt,AdrBalUSD100Cnt,AdrBalUSD100KCnt,AdrBalUSD10Cnt,AdrBalUSD10KCnt,AdrBalUSD10MCnt,AdrBalUSD1Cnt,AdrBalUSD1KCnt,AdrBalUSD1MCnt,AssetEODCompletionTime,BlkCnt,BlkSizeMeanByte,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktFFUSD,CapRealUSD,DiffLast,DiffMean,...,SplyAdrBal1in10M,SplyAdrBal1in1B,SplyAdrBal1in1K,SplyAdrBal1in1M,SplyAdrBalNtv0.001,SplyAdrBalNtv0.01,SplyAdrBalNtv0.1,SplyAdrBalNtv1,SplyAdrBalNtv10,SplyAdrBalNtv100,SplyAdrBalNtv100K,SplyAdrBalNtv10K,SplyAdrBalNtv1K,SplyAdrBalNtv1M,SplyAdrBalUSD1,SplyAdrBalUSD10,SplyAdrBalUSD100,SplyAdrBalUSD100K,SplyAdrBalUSD10K,SplyAdrBalUSD10M,SplyAdrBalUSD1K,SplyAdrBalUSD1M,SplyAdrTop100,SplyAdrTop10Pct,SplyAdrTop1Pct,SplyCur,SplyExpFut10yr,SplyFF,TxCnt,TxCntSec,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d
0,2016-07-20,21680.0,5491.0,63999.0,139156.0,956.0,34371.0,90814.0,102.0,15524.0,251206.0,168782.0,135046.0,86861.0,14239.0,88.0,32387.0,831.0,59113.0,4966.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,3591.0,2011.762462,NaN,NaN,NaN,NaN,NaN,NaN,1.093854e+13,5.860014e+13,...,8.208438e+07,8.216821e+07,4.973673e+07,8.152539e+07,8.216977e+07,8.216965e+07,8.216783e+07,8.215578e+07,8.206606e+07,8.140725e+07,4.845869e+07,6.689327e+07,7.844824e+07,2.895121e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.956878e+07,8.195765e+07,7.470971e+07,8.216978e+07,1.372007e+08,1.081479e+07,37562.0,0.434745,30374.0,1.840795e+07,NaN,899.502488,NaN,1.000024,NaN,0.332500,NaN,NaN
1,2016-07-21,17037.0,5471.0,64419.0,140717.0,960.0,34392.0,91580.0,103.0,15515.0,252849.0,170404.0,136632.0,87615.0,14229.0,88.0,32394.0,832.0,59515.0,4954.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,1799.0,2837.280712,NaN,NaN,NaN,NaN,NaN,NaN,5.616330e+11,1.271276e+12,...,8.209269e+07,8.217744e+07,4.972084e+07,8.153163e+07,8.217903e+07,8.217892e+07,8.217706e+07,8.216489e+07,8.207422e+07,8.141345e+07,4.835555e+07,6.692544e+07,7.845158e+07,2.895121e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.946770e+07,8.196906e+07,7.477675e+07,8.217905e+07,1.223244e+08,1.323342e+07,30035.0,0.347627,24992.0,7.504155e+05,NaN,34.523939,NaN,0.192971,NaN,0.342962,NaN,NaN
2,2016-07-22,13208.0,5497.0,64617.0,140936.0,960.0,34619.0,91833.0,101.0,15622.0,253504.0,170681.0,136840.0,87836.0,14338.0,87.0,32635.0,834.0,59684.0,4970.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,5669.0,1107.125066,NaN,NaN,NaN,NaN,NaN,NaN,4.402919e+11,4.862804e+11,...,8.212352e+07,8.220802e+07,4.947728e+07,8.155725e+07,8.220962e+07,8.220950e+07,8.220765e+07,8.219544e+07,8.210517e+07,8.143916e+07,4.819902e+07,6.686495e+07,7.844524e+07,2.895121e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.939334e+07,8.199628e+07,7.475535e+07,8.220963e+07,1.306924e+08,1.408255e+07,19557.0,0.226354,15504.0,1.642068e+06,NaN,105.987807,NaN,0.268690,NaN,0.362823,NaN,NaN
3,2016-07-23,9838.0,5499.0,64505.0,140990.0,960.0,34529.0,91857.0,103.0,15600.0,254059.0,170946.0,136895.0,87753.0,14331.0,87.0,32561.0,831.0,59584.0,4975.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,5503.0,961.825186,NaN,NaN,NaN,NaN,NaN,NaN,3.099510e+11,3.716707e+11,...,8.215335e+07,8.223768e+07,4.971997e+07,8.158876e+07,8.223928e+07,8.223916e+07,8.223729e+07,8.222508e+07,8.213515e+07,8.147199e+07,4.826322e+07,6.685387e+07,7.847848e+07,2.895121e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.946897e+07,8.202888e+07,7.478552e+07,8.223929e+07,1.359277e+08,1.815243e+07,14304.0,0.165556,10534.0,4.321988e+06,NaN,414.256390,NaN,0.112851,NaN,0.415754,NaN,NaN
4,2016-07-24,14910.0,5304.0,62722.0,138707.0,910.0,33629.0,89679.0,98.0,15048.0,255994.0,170788.0,134572.0,85580.0,13809.0,83.0,31706.0,781.0,57745.0,4800.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.61

In [ ]:
### change to dateime
df_etc['time']=pd.to_datetime(df_etc['time'])
df_etc.dtypes

time                datetime64[ns]
AdrActCnt                  float64
AdrBal1in100KCnt           float64
AdrBal1in100MCnt           float64
AdrBal1in10BCnt            float64
                         ...      
TxTfrValMedNtv             float64
TxTfrValMedUSD             float64
VelCur1yr                  float64
VtyDayRet180d              float64
VtyDayRet30d               float64
Length: 134, dtype: object

In [ ]:
df_etc['Type'] = 'etc'
df_etc.head()

,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,AdrBalNtv0.001Cnt,AdrBalNtv0.01Cnt,AdrBalNtv0.1Cnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,AdrBalNtv1Cnt,AdrBalNtv1KCnt,AdrBalNtv1MCnt,AdrBalUSD100Cnt,AdrBalUSD100KCnt,AdrBalUSD10Cnt,AdrBalUSD10KCnt,AdrBalUSD10MCnt,AdrBalUSD1Cnt,AdrBalUSD1KCnt,AdrBalUSD1MCnt,AssetEODCompletionTime,BlkCnt,BlkSizeMeanByte,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktFFUSD,CapRealUSD,DiffLast,DiffMean,...,SplyAdrBal1in1B,SplyAdrBal1in1K,SplyAdrBal1in1M,SplyAdrBalNtv0.001,SplyAdrBalNtv0.01,SplyAdrBalNtv0.1,SplyAdrBalNtv1,SplyAdrBalNtv10,SplyAdrBalNtv100,SplyAdrBalNtv100K,SplyAdrBalNtv10K,SplyAdrBalNtv1K,SplyAdrBalNtv1M,SplyAdrBalUSD1,SplyAdrBalUSD10,SplyAdrBalUSD100,SplyAdrBalUSD100K,SplyAdrBalUSD10K,SplyAdrBalUSD10M,SplyAdrBalUSD1K,SplyAdrBalUSD1M,SplyAdrTop100,SplyAdrTop10Pct,SplyAdrTop1Pct,SplyCur,SplyExpFut10yr,SplyFF,TxCnt,TxCntSec,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,Type
0,2016-07-20,21680.0,5491.0,63999.0,139156.0,956.0,34371.0,90814.0,102.0,15524.0,251206.0,168782.0,135046.0,86861.0,14239.0,88.0,32387.0,831.0,59113.0,4966.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,3591.0,2011.762462,NaN,NaN,NaN,NaN,NaN,NaN,1.093854e+13,5.860014e+13,...,8.216821e+07,4.973673e+07,8.152539e+07,8.216977e+07,8.216965e+07,8.216783e+07,8.215578e+07,8.206606e+07,8.140725e+07,4.845869e+07,6.689327e+07,7.844824e+07,2.895121e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.956878e+07,8.195765e+07,7.470971e+07,8.216978e+07,1.372007e+08,1.081479e+07,37562.0,0.434745,30374.0,1.840795e+07,NaN,899.502488,NaN,1.000024,NaN,0.332500,NaN,NaN,etc
1,2016-07-21,17037.0,5471.0,64419.0,140717.0,960.0,34392.0,91580.0,103.0,15515.0,252849.0,170404.0,136632.0,87615.0,14229.0,88.0,32394.0,832.0,59515.0,4954.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,1799.0,2837.280712,NaN,NaN,NaN,NaN,NaN,NaN,5.616330e+11,1.271276e+12,...,8.217744e+07,4.972084e+07,8.153163e+07,8.217903e+07,8.217892e+07,8.217706e+07,8.216489e+07,8.207422e+07,8.141345e+07,4.835555e+07,6.692544e+07,7.845158e+07,2.895121e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.946770e+07,8.196906e+07,7.477675e+07,8.217905e+07,1.223244e+08,1.323342e+07,30035.0,0.347627,24992.0,7.504155e+05,NaN,34.523939,NaN,0.192971,NaN,0.342962,NaN,NaN,etc
2,2016-07-22,13208.0,5497.0,64617.0,140936.0,960.0,34619.0,91833.0,101.0,15622.0,253504.0,170681.0,136840.0,87836.0,14338.0,87.0,32635.0,834.0,59684.0,4970.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,5669.0,1107.125066,NaN,NaN,NaN,NaN,NaN,NaN,4.402919e+11,4.862804e+11,...,8.220802e+07,4.947728e+07,8.155725e+07,8.220962e+07,8.220950e+07,8.220765e+07,8.219544e+07,8.210517e+07,8.143916e+07,4.819902e+07,6.686495e+07,7.844524e+07,2.895121e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.939334e+07,8.199628e+07,7.475535e+07,8.220963e+07,1.306924e+08,1.408255e+07,19557.0,0.226354,15504.0,1.642068e+06,NaN,105.987807,NaN,0.268690,NaN,0.362823,NaN,NaN,etc
3,2016-07-23,9838.0,5499.0,64505.0,140990.0,960.0,34529.0,91857.0,103.0,15600.0,254059.0,170946.0,136895.0,87753.0,14331.0,87.0,32561.0,831.0,59584.0,4975.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,5503.0,961.825186,NaN,NaN,NaN,NaN,NaN,NaN,3.099510e+11,3.716707e+11,...,8.223768e+07,4.971997e+07,8.158876e+07,8.223928e+07,8.223916e+07,8.223729e+07,8.222508e+07,8.213515e+07,8.147199e+07,4.826322e+07,6.685387e+07,7.847848e+07,2.895121e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.946897e+07,8.202888e+07,7.478552e+07,8.223929e+07,1.359277e+08,1.815243e+07,14304.0,0.165556,10534.0,4.321988e+06,NaN,414.256390,NaN,0.112851,NaN,0.415754,NaN,NaN,etc
4,2016-07-24,14910.0,5304.0,62722.0,138707.0,910.0,33629.0,89679.0,98.0,15048.0,255994.0,170788.0,134572.0,85580.0,13809.0,83.0,31706.0,781.0,57745.0,4800.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.611240e+09,9388.0,1010.318705,NaN,NaN,NaN,NaN,NaN,

# Regression Discontinguity

https://youtu.be/TfKwgGT2fSM

### create the identifier varaible for Dao Hack

In [ ]:
from datetime import date

In [ ]:
df_eth ['After_DaoHack'] =df_eth['time'].apply(lambda x: 1 if x>= date(2016,7,20) else 0)
df_eth[df_eth.time.dt.date>=date(2016,7,18)].head()

In [ ]:
df_eth['days']=df_eth.index
df_eth.head()

,index,time,PriceUSD,TxTfrValAdjUSD,FeeMedUSD,RevUSD,GasLmtTxMean,Type,AdrBalCnt,days,After_DaoHack
0,361,2016-07-25,13.840295,1.100270e+08,0.005813,443318.388276,116866.529623,eth,260209.0,0,1
1,362,2016-07-26,11.963285,1.962552e+08,0.005025,368654.474065,106522.289666,eth,261319.0,1,1
2,363,2016-07-27,13.061564,1.251180e+08,0.005657,430816.779943,126615.358103,eth,261639.0,2,1
3,364,2016-07-28,12.888203,8.846049e+07,0.005788,417811.961067,133514.917908,eth,263622.0,3,1
4,365,2016-07-29,12.878013,5.242804e+07,0.005409,411903.974753,139836.829202,eth,267380.0,4,1


### normalize the data

In [ ]:
df_eth['days'][df_eth.time.dt.date==date(2016,7,20)]

356    356
Name: days, dtype: int64

In [ ]:
df_eth['days']=df_eth['days']-356
df_eth[df_eth.time.dt.date>=date(2016,7,18)].head()

,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,AdrBalNtv0.001Cnt,AdrBalNtv0.01Cnt,AdrBalNtv0.1Cnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,AdrBalNtv1Cnt,AdrBalNtv1KCnt,AdrBalNtv1MCnt,AdrBalUSD100Cnt,AdrBalUSD100KCnt,AdrBalUSD10Cnt,AdrBalUSD10KCnt,AdrBalUSD10MCnt,AdrBalUSD1Cnt,AdrBalUSD1KCnt,AdrBalUSD1MCnt,AssetEODCompletionTime,BlkCnt,BlkSizeMeanByte,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktFFUSD,CapRealUSD,DiffLast,DiffMean,...,SplyAdrBal1in1M,SplyAdrBalNtv0.001,SplyAdrBalNtv0.01,SplyAdrBalNtv0.1,SplyAdrBalNtv1,SplyAdrBalNtv10,SplyAdrBalNtv100,SplyAdrBalNtv100K,SplyAdrBalNtv10K,SplyAdrBalNtv1K,SplyAdrBalNtv1M,SplyAdrBalUSD1,SplyAdrBalUSD10,SplyAdrBalUSD100,SplyAdrBalUSD100K,SplyAdrBalUSD10K,SplyAdrBalUSD10M,SplyAdrBalUSD1K,SplyAdrBalUSD1M,SplyAdrTop100,SplyAdrTop10Pct,SplyAdrTop1Pct,SplyCur,SplyExpFut10yr,SplyFF,TxCnt,TxCntSec,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,Type,After_DaoHack,days
354,2016-07-18,18988.0,6564.0,72880.0,148000.0,1112.0,40981.0,100043.0,120.0,19127.0,259306.0,177701.0,143793.0,96228.0,17524.0,100.0,38727.0,967.0,67755.0,5908.0,5.0,39998.0,1046.0,71788.0,6295.0,5.0,98954.0,18441.0,109.0,1.614335e+09,6109.0,1397.454248,6.477075e+08,1.452459,1.409655,9.100982e+08,8.832774e+08,6.265913e+08,6.259702e+13,5.977354e+13,...,8.137529e+07,8.211897e+07,8.211886e+07,8.211708e+07,8.210459e+07,8.200494e+07,8.122778e+07,4.205752e+07,6.372227e+07,7.750533e+07,1.955986e+07,8.211734e+07,8.210854e+07,8.201722e+07,6.447809e+07,7.788077e+07,1.955986e+07,8.131612e+07,4.293459e+07,4.205752e+07,8.175258e+07,7.210092e+07,8.211899e+07,1.999045e+08,7.969892e+07,39198.0,0.453681,36607.0,2.028033e+06,2.247604e+07,79.939170,885.940002,1.003258,11.118781,4.614087,0.083592,0.056129,eth,0,-2
355,2016-07-19,26614.0,5492.0,63770.0,138524.0,965.0,34424.0,90180.0,103.0,15552.0,250201.0,168167.0,134316.0,86580.0,14248.0,89.0,32420.0,840.0,58903.0,4971.0,6.0,34092.0,949.0,63394.0,5404.0,6.0,89901.0,15325.0,100.0,1.614335e+09,5970.0,1501.899330,6.827144e+08,1.485046,1.441299,9.591615e+08,9.309057e+08,6.458799e+08,5.982954e+13,6.039296e+13,...,8.150794e+07,8.215075e+07,8.215063e+07,8.214885e+07,8.213685e+07,8.204729e+07,8.138800e+07,4.808518e+07,6.688142e+07,7.843026e+07,2.825250e+07,8.214917e+07,8.214117e+07,8.206300e+07,6.789685e+07,7.883803e+07,2.825250e+07,8.148892e+07,4.910645e+07,4.910645e+07,8.193610e+07,7.465082e+07,8.215076e+07,1.999691e+08,7.973069e+07,44184.0,0.511389,42327.0,9.600101e+06,1.120872e+08,230.947681,2696.458549,1.046876,12.222931,4.731295,0.083639,0.054430,eth,0,-1
356,2016-07-20,25428.0,5730.0,64421.0,139758.0,1019.0,35008.0,91222.0,107.0,16043.0,251941.0,169270.0,135660.0,87332.0,14744.0,94.0,33040.0,879.0,59617.0,5190.0,5.0,35294.0,1066.0,64712.0,5845.0,7.0,91441.0,16273.0,110.0,1.614335e+09,5946.0,1717.926001,7.189636e+08,1.580868,1.534315,1.027572e+09,9.973125e+08,6.500050e+08,5.707466e+13,5.892911e+13,...,8.153395e+07,8.218266e+07,8.218255e+07,8.218073e+07,8.216876e+07,8.207941e+07,8.141458e+07,4.677036e+07,6.616255e+07,7.834983e+07,2.414082e+07,8.218111e+07,8.217357e+07,8.209989e+07,6.783351e+07,7.894194e+07,2.596738e+07,8.155254e+07,4.820940e+07,4.735547e+07,8.195819e+07,7.418751e+07,8.218267e+07,1.998917e+08,7.976260e+07,50631.0,0.586007,43432.0,3.111432e+07,3.890382e+08,933.138829,11667.510738,0.906124,11.329737,5.222604,0.083747,0.055036,eth,1,0
357,2016-07-21,22477.0,5866.0,64800.0,140064.0,1037.0,35418.0,91196.0,107.0,16414.0,253142.0,169588.0,135996.0,87618.0,15112.0,93.0,33476.0,895.0,60026.0,5323.0,5.0,35893.0,1099.0,65320.0,6029.0,7.0,91568.0,16787.0,112.0,1.614335e+09,6154.0,1522.185570,7.340349e+08,1.596223,1.549237,1.047885e+09,1.017039e+09,6.564775e+08,6.196587e+13,5.976263e+13,...,8.156106e+07,8.221545e+07,8.221533e+07,8.221352e+07,8.220161e+07,8.211220e+07,8.144

## create the regression function

https://www.statsmodels.org/stable/index.html

In [ ]:
 import statsmodels.api as sm
 import statsmodels.formula.api as smf

In [ ]:
results = smf.ols('TxTfrValAdjUSD ~ 1+ After_DaoHack +days+AdrBalCnt ', data=df_eth).fit()

In [ ]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         TxTfrValAdjUSD   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     267.8
Date:                Sun, 27 Jun 2021   Prob (F-statistic):          1.20e-147
Time:                        16:15:00   Log-Likelihood:                -50256.
No. Observations:                2150   AIC:                         1.005e+05
Df Residuals:                    2146   BIC:                         1.005e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.992e+09   2.44e+08     -8.178

In [ ]:
results = smf.ols('TxTfrValAdjUSD ~ 1+ After_DaoHack + days +After_DaoHack*days +AdrBalCnt', data=df_eth).fit()

In [ ]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         TxTfrValAdjUSD   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     217.4
Date:                Sun, 27 Jun 2021   Prob (F-statistic):          9.04e-157
Time:                        16:15:00   Log-Likelihood:                -50232.
No. Observations:                2150   AIC:                         1.005e+05
Df Residuals:                    2145   BIC:                         1.005e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           -8.01e+07   3.65

# Difference in Difference

https://towardsdatascience.com/causal-inference-101-difference-in-differences-1fbbb0f55e85

### prepare the datasets

In [ ]:
Columns =["time","PriceUSD","TxTfrValAdjUSD","FeeMedUSD","RevUSD","GasLmtTxMean","Type","AdrBalCnt"]
df_eth =df_eth[Columns]
df_etc =df_etc[Columns]

In [ ]:
df_etc =df_etc.dropna()
df_etc.head()

,time,PriceUSD,TxTfrValAdjUSD,FeeMedUSD,RevUSD,GasLmtTxMean,Type,AdrBalCnt
5,2016-07-25,0.609005,3.988031e+06,0.000294,19053.076289,128512.450456,etc,258077.0
6,2016-07-26,2.550458,2.138461e+07,0.001242,117166.149612,106473.058865,etc,259978.0
7,2016-07-27,1.619800,1.597548e+07,0.000770,53581.634999,121838.776100,etc,260012.0
8,2016-07-28,1.727024,2.210299e+07,0.000879,57859.069868,118155.032968,etc,261181.0
9,2016-07-29,1.649872,1.257784e+07,0.001039,58069.745834,108353.172803,etc,262251.0


In [ ]:
df_eth =df_eth[df_eth.time.dt.date>=date(2016,7,25)]
df_etc =df_etc[df_etc.time.dt.date>=date(2016,7,25)]

In [ ]:
df_eth=df_eth.reset_index()
df_etc=df_etc.reset_index()

In [ ]:
df_eth["days"]=df_eth.index
df_eth.head()

In [ ]:
df_etc["days"]=df_etc.index
df_etc.head()

In [ ]:
df = pd.concat([df_eth, df_etc], axis=0)
df.head()

In [ ]:
df["etc"]=df["Type"].apply(lambda x: 1 if x=="etc" else 0)
df.head()

,index,time,PriceUSD,TxTfrValAdjUSD,FeeMedUSD,RevUSD,GasLmtTxMean,Type,AdrBalCnt,days,etc
0,361,2016-07-25,13.840295,1.100270e+08,0.005813,443318.388276,116866.529623,eth,260209.0,0,0
1,362,2016-07-26,11.963285,1.962552e+08,0.005025,368654.474065,106522.289666,eth,261319.0,1,0
2,363,2016-07-27,13.061564,1.251180e+08,0.005657,430816.779943,126615.358103,eth,261639.0,2,0
3,364,2016-07-28,12.888203,8.846049e+07,0.005788,417811.961067,133514.917908,eth,263622.0,3,0
4,365,2016-07-29,12.878013,5.242804e+07,0.005409,411903.974753,139836.829202,eth,267380.0,4,0


### run the regression

Treatment: ETC hard-fork

In [ ]:
 import statsmodels.api as sm
 import statsmodels.formula.api as smf

In [ ]:
results = smf.ols('TxTfrValAdjUSD ~ 1+ etc +days+AdrBalCnt+FeeMedUSD+RevUSD+GasLmtTxMean', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         TxTfrValAdjUSD   R-squared:                       0.693
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     1316.
Date:                Sun, 27 Jun 2021   Prob (F-statistic):               0.00
Time:                        16:15:00   Log-Likelihood:                -79764.
No. Observations:                3510   AIC:                         1.595e+05
Df Residuals:                    3503   BIC:                         1.596e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -7.185e+08   8.45e+07     -8.503   

In [ ]:
results = smf.ols('TxTfrValAdjUSD ~ 1+ etc +days +etc*days+AdrBalCnt+FeeMedUSD+RevUSD+GasLmtTxMean', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         TxTfrValAdjUSD   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     1139.
Date:                Sun, 27 Jun 2021   Prob (F-statistic):               0.00
Time:                        16:15:00   Log-Likelihood:                -79752.
No. Observations:                3510   AIC:                         1.595e+05
Df Residuals:                    3502   BIC:                         1.596e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -4.937e+07    1.6e+08     -0.309   